# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/cities.csv", encoding="utf-8")
weather_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,dikson,56,RU,1616122375,97,73.5069,80.5464,-32.02,3.95
1,1,ostrovnoy,99,RU,1616122471,84,68.0531,39.5131,-10.47,2.64
2,2,ponta do sol,20,PT,1616122401,67,32.6667,-17.1000,16.67,4.12
3,3,ushuaia,40,AR,1616122295,71,-54.8000,-68.3000,8.00,2.06
4,4,san cristobal,75,VE,1616122448,100,7.7669,-72.2250,24.00,2.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [62]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Convert to float
weights = weather_df["Humidity"].astype(float)

max_intensity = weather_df["Humidity"].max()

In [63]:
# Plot Heatmap
fig = gmaps.figure(zoom_level=2, center=(0,0))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=max_intensity, 
                                 point_radius=4)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
hotel_df = weather_df.loc[(weather_df["Cloudiness"] == 0) & 
                              (weather_df["Humidity"] < 50) & 
                              (weather_df["Wind Speed"] < 5) & 
                              (weather_df["Max Temp"] < 25) & 
                              (weather_df["Max Temp"] > 15), :].copy()

In [7]:
hotel_df = hotel_df.dropna(how='any')

In [8]:
hotel_df.shape

(13, 10)

In [9]:
hotel_df.describe()

,City_ID,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
count,13.000000,13.0,1.300000e+01,13.000000,13.000000,13.000000,13.000000,13.000000
mean,300.615385,0.0,1.616123e+09,29.615385,19.446977,39.528485,20.844615,2.435385
std,172.275718,0.0,1.938240e+02,10.797673,18.884023,50.848359,2.433498,1.620101
min,48.000000,0.0,1.616122e+09,13.000000,-40.813500,-62.996700,15.110000,0.260000
25%,181.000000,0.0,1.616122e+09,24.000000,20.502200,-3.983600,20.000000,1.540000
50%,263.000000,0.0,1.616123e+09,31.000000,22.690600,54.092400,21.480000,2.060000
75%,467.000000,0.0,1.616123e+09,37.000000,29.121200,71.683300,22.410000,4.160000
max,588.000000,0.0,1.616123e+09,45.000000,34.685100,135.804900,23.120000,4.700000


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df["Hotel Name"] = ""

In [56]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "",
    "keyword": "hotel",    
    "radius": 5000,
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    params['location'] = str(row['Lat']) + "," + str(row['Lng'])
    response = requests.get(base_url, params=params).json()

    print(f"Retrieving Results for Index {index}.")
        
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[index, 'Hotel Name'] = None      
        
    print("------------")
  

Retrieving Results for Index 48.
Closest hotel is Bab Sahara.
------------
Retrieving Results for Index 145.
Closest hotel is Star Emirates garden villas.
------------
Retrieving Results for Index 176.
Missing field/result... skipping.
------------
Retrieving Results for Index 181.
Closest hotel is HP Red Sea Hotel.
------------
Retrieving Results for Index 190.
Missing field/result... skipping.
------------
Retrieving Results for Index 233.
Closest hotel is Antique Inn - Nizwa.
------------
Retrieving Results for Index 263.
Closest hotel is Onyado Nono.
------------
Retrieving Results for Index 267.
Closest hotel is تفضل بيت ضيافة Geust house tafadal.
------------
Retrieving Results for Index 291.
Missing field/result... skipping.
------------
Retrieving Results for Index 467.
Missing field/result... skipping.
------------
Retrieving Results for Index 486.
Closest hotel is Masha ALLAH Restaurant.
------------
Retrieving Results for Index 573.
Closest hotel is Casa Crespo Hotel Boutiqu

In [57]:
hotel_df = hotel_df.dropna(how='any')

In [58]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
48,48,atar,0,MR,1616122485,13,20.5169,-13.0499,20.90,1.64,Bab Sahara
145,145,salalah,0,OM,1616122415,43,17.0151,54.0924,22.00,1.54,Star Emirates garden villas
181,181,umm lajj,0,SA,1616122623,45,25.0213,37.2685,21.48,4.62,HP Red Sea Hotel
233,233,nizwa,0,OM,1616122824,24,22.9333,57.5333,23.00,0.39,Antique Inn - Nizwa
263,263,nara,0,JP,1616122826,39,34.6851,135.8049,20.00,2.06,Onyado Nono
267,267,ibra,0,OM,1616122833,31,22.6906,58.5334,22.19,0.26,تفضل بيت ضيافة Geust house tafadal
486,486,yazman,0,PK,1616122899,31,29.1212,71.7446,23.08,4.31,Masha ALLAH Restaurant
573,573,viedma,0,AR,1616122414,37,-40.8135,-62.9967,15.11,4.70,Casa Crespo Hotel Boutique
588,588,bahawalpur,0,PK,1616122796,36,29.4000,71.6833,22.41,4.16,The Grand Regency


In [65]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [66]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))